<a href="https://colab.research.google.com/github/Huxwell/ColabNNs/blob/main/cats_n_dogs_generate_multicategory_dataset_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Just viewing my options
What was done:
*   8 variants of concat 
*   all images resized to 224x224





In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
import glob
from PIL import Image
import random # do random.seed(13) before every shuffle. order of shuffle() execution changes results order.

In [ ]:
dataset_dir = '/content/drive/MyDrive/cats_n_dogs_unsure/dogs-vs-cats-redux-kernels-edition'
os.listdir(dataset_dir)

In [ ]:
os.makedirs('local_data/', exist_ok=True)
train_dir = 'local_data/train'

In [ ]:
!pwd
!ls /content/local_data/train


In [ ]:
with zipfile.ZipFile(os.path.join(dataset_dir, 'train.zip')) as train_zip:
    train_zip.extractall('local_data')

In [ ]:
os.listdir(train_dir)[:5]

In [ ]:
cats_list = sorted(glob.glob(os.path.join(train_dir,'cat*.jpg')))
dogs_list = sorted(glob.glob(os.path.join(train_dir,'dog*.jpg')))
print(len(cats_list))
print(len(dogs_list))
random.seed(13)
random.shuffle(cats_list)
random.seed(13) #multiple seed() executions are on purpose.
random.shuffle(dogs_list)
print(cats_list[:3])
print(dogs_list[:3])
# some sanity check to make sure no uncontrolled randomness beyond this point
assert cats_list[2] == 'local_data/train/cat.801.jpg'
assert dogs_list[2] == 'local_data/train/dog.801.jpg'
assert len(cats_list) == len(dogs_list)


## This time I have to shuffle cats list and dogs list separately
So that each cat photo can be matched with a dog photo.

In [ ]:
test_cats = cats_list[10000:]
test_dogs = dogs_list[10000:]
print(len(test_cats))
print(len(test_dogs))

random.seed(13)
random.shuffle(test_cats)

random.seed(1313) # you get the same indeces if you do seed(13) again !!! order doesn't depend on content !!!
random.shuffle(test_dogs)

print("test_cats",test_cats[:9])
assert test_cats[0] == 'local_data/train/cat.4660.jpg'

print("test_dogs",test_dogs[:9])
assert test_dogs[0] == 'local_data/train/dog.1383.jpg'

In [ ]:
# test_list = cats_list[10000:] + dogs_list[10000:]
# print(len(test_list))

# random.seed(13)
# random.shuffle(test_list)
# print("test",test_list[:9])
# assert test_list[4] == 'local_data/train/dog.6485.jpg'

# finished here: todo: actual generation

In [ ]:
np.random.seed(1313131313)
random_idx = np.random.randint(1,len(test_cats),size=10)
print(random_idx)
fig = plt.figure(figsize=(20., 10.))

for i, img_idx in enumerate(random_idx):
    ax = fig.add_subplot(2,5,i+1)
    img = Image.open(test_cats[img_idx])
    plt.imshow(img)
    i+=1

plt.axis('off')
plt.show()

In [ ]:
np.random.seed(1313131313)
random_idx = np.random.randint(1,len(test_dogs),size=10)
print(random_idx)
fig = plt.figure(figsize=(20., 10.))

for i, img_idx in enumerate(random_idx):
    ax = fig.add_subplot(2,5,i+1)
    img = Image.open(test_dogs[img_idx])
    plt.imshow(img)
    i+=1

plt.axis('off')
plt.show()

In [ ]:
def crop_center(img, cropx, cropy):
    y, x, *_ = img.shape
    startx = x // 2 - (cropx // 2)
    starty = y // 2 - (cropy // 2)    
    return img[starty:starty + cropy, startx:startx + cropx, ...]
def concat_variants(img1, img2, variant=0): #not using match case, because this is python 3.8 not 3.10
    if variant == 0:
      return np.concatenate((img1,img2), axis=0)
    elif variant == 1:
      return np.concatenate((img1,img2), axis=1)
    elif variant == 2:
      return np.concatenate((img2,img1), axis=0)
    elif variant == 3:
      return np.concatenate((img2,img1), axis=1)
    elif variant == 4:
      return np.concatenate((img2,img1), axis=0)
    elif variant == 5:
      return np.concatenate((img2,img1), axis=1)
    elif variant == 6:
      return np.concatenate((img1,img2), axis=0)
    elif variant == 7:
      return np.concatenate((img1,img2), axis=1)


In [ ]:
np.random.seed(13131313)
random_idx = np.random.randint(1,len(test_dogs),size=8)
print(random_idx)
fig = plt.figure(figsize=(20., 10.))

for i, img_idx in enumerate(random_idx):
    ax = fig.add_subplot(2,4,i+1)
    img = Image.open(test_dogs[img_idx]).resize((224,224))
    img2 = Image.open(test_cats[img_idx]).resize((224,224))
    img_concat = concat_variants(img, img2, i%8)
    plt.imshow(crop_center(img_concat, 224,224))
    i+=1

plt.axis('off')
plt.show()

In [ ]:
!rm -rvf generated/

In [ ]:
os.makedirs('generated/', exist_ok=True)


for idx, (img, img2) in enumerate(zip(test_dogs, test_cats)):
    print(img,img2)
    dog_path = img
    img = Image.open(img).resize((224,224))
    img2 = Image.open(img2).resize((224,224))
    img_concat = concat_variants(img, img2, i%8)
    img_concat = crop_center(img_concat, 224,224)
    
    outpath = f"cat_dog_50_50_8concats_{idx}_224x224resize.jpg"
    plt.imsave(f"generated/{outpath}", img_concat)

In [ ]:
!ls generated/

In [ ]:
!rm -rv /content/drive/MyDrive/cat_dogs_8_contats_50_50_from_test
!mkdir /content/drive/MyDrive/cat_dogs_8_contats_50_50_from_test
!cp -rv generated/* /content/drive/MyDrive/cat_dogs_8_contats_50_50_from_test/
!rm -r generated

In [ ]:
!ls /content/drive/MyDrive/cat_dogs_8_contats_50_50_from_test | wc -l

In [ ]:
!zip -rj cat_dogs_8_contats_50_50_from_test_fix.zip /content/drive/MyDrive/cat_dogs_8_contats_50_50_from_test 

In [ ]:
!cp -rv cat_dogs_8_contats_50_50_from_test_fix.zip /content/drive/MyDrive/cat_dogs_8_contats_50_50_from_test.zip

In [ ]:
!rm -rv /content/drive/MyDrive/cat_dogs_8_contats_50_50_from_test 